In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [2]:
import dspy
dspy.settings.configure(lm=dspy.LM("openai/gpt-4o-mini"))

In [3]:
class SentimentClassifier(dspy.Signature):
    """Classify the sentiment of a text."""
  
    text: str = dspy.InputField(desc="input text to classify sentiment")
    sentiment: int = dspy.OutputField(
        desc="sentiment, the higher the more positive", ge=0, le=10
    )

In [4]:
str_signature = dspy.make_signature("text -> sentiment")

In [5]:
predict = dspy.Predict(SentimentClassifier) 

In [6]:
output = predict(text="I am feeling pretty happy!")
print(output)

Prediction(
    sentiment=8
)


In [7]:
print(f"The sentiment is: {output.sentiment}")
print(f"The sentiment is: {output['sentiment']}")

The sentiment is: 8
The sentiment is: 8


In [8]:
dspy.inspect_history(n=1)





[2025-09-17T22:30:03.524154]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with the corresponding output fields, starting with the field `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## sentiment ## ]]
8

[[ ## completed ## ]]







In [9]:
cot = dspy.ChainOfThought(SentimentClassifier)

output = cot(text="I am feeling pretty happy!")
print(output)

Prediction(
    reasoning='The text expresses a positive emotion, specifically happiness. The use of the word "happy" indicates a strong positive sentiment. Therefore, the sentiment score is high.',
    sentiment=8
)


In [10]:
dspy.inspect_history(n=1)





[2025-09-17T22:32:06.150757]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str): 
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## sentiment ## ]]
{sentiment}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am feeling pretty happy!

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## sentiment ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
Th

In [11]:
dspy.configure(adapter=dspy.JSONAdapter())

In [12]:
print(cot(text="I am feeling pretty happy!"))
dspy.inspect_history(n=1)

Prediction(
    reasoning="The text expresses a positive emotion, specifically happiness, which indicates a favorable sentiment. The use of the word 'happy' suggests a strong positive feeling.",
    sentiment=8
)




[2025-09-17T22:32:42.361914]

System message:

Your input fields are:
1. `text` (str): input text to classify sentiment
Your output fields are:
1. `reasoning` (str): 
2. `sentiment` (int): sentiment, the higher the more positive
Constraints: greater than or equal to: 0, less than or equal to: 10
All interactions will be structured in the following way, with the appropriate values filled in.

Inputs will have the following structure:

[[ ## text ## ]]
{text}

Outputs will be a JSON object with the following fields.

{
  "reasoning": "{reasoning}",
  "sentiment": "{sentiment}        # note: the value you produce must be a single int value"
}
In adhering to this structure, your objective is: 
        Classify the sentiment of a text.


User message:

[[ ## text ## ]]
I am fee

In [13]:
class QuestionGenerator(dspy.Signature):
    """
    Generate a yes or no question in order to guess the celebrity name in users' mind. 
    You can ask in general or directly guess the name if you think the signal is enough.
    You should never ask the same question in the past_questions.
    """
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")
    new_question: str = dspy.OutputField(desc="new question that can help narrow down the celebrity name")
    guess_made: bool = dspy.OutputField(desc="If the new_question is the celebrity name guess, set to True, if it is still a general question set to False")


class Reflection(dspy.Signature):
    """Provide reflection on the guessing process"""
    correct_celebrity_name: str = dspy.InputField(desc="the celebrity name in user's mind")
    final_guessor_question: str = dspy.InputField(desc="the final guess or question LM made")
    past_questions: list[str] = dspy.InputField(desc="past questions asked")
    past_answers: list[bool] = dspy.InputField(desc="past answers")

    reflection: str = dspy.OutputField(
        desc="reflection on the guessing process, including what was done well and what can be improved"
    )

def ask(prompt, valid_responses=("y", "n")):
    while True:
        response = input(f"{prompt} ({'/'.join(valid_responses)}): ").strip().lower()
        if response in valid_responses:
            return response
        print(f"Please enter one of: {', '.join(valid_responses)}")

class CelebrityGuess(dspy.Module):
    def __init__(self, max_tries=10):
        super().__init__()

        self.question_generator = dspy.ChainOfThought(QuestionGenerator)
        self.reflection = dspy.ChainOfThought(Reflection)

        self.max_tries = 20

    def forward(self):
        celebrity_name = input("Please think of a celebrity name, once you are ready, type the name and press enter...")
        past_questions = []
        past_answers = []

        correct_guess = False

        for i in range(self.max_tries):
            question = self.question_generator(
                past_questions=past_questions,
                past_answers=past_answers,
            )
            answer = ask(f"{question.new_question}").lower() == "y"
            past_questions.append(question.new_question)
            past_answers.append(answer)

            if question.guess_made and answer:
                correct_guess = True
                break

        if correct_guess:
            print("Yay! I got it right!")
        else:
            print("Oops, I couldn't guess it right.")

        reflection = self.reflection(
            correct_celebrity_name=celebrity_name,
            final_guessor_question=question.new_question,
            past_questions=past_questions,
            past_answers=past_answers,
        )
        print(reflection.reflection)


In [14]:
celebrity_guess = CelebrityGuess()

In [15]:
celebrity_guess()

Please enter one of: y, n
Please enter one of: y, n


KeyboardInterrupt: Interrupted by user